In [4]:
import findspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import substring

spark = SparkSession.builder.appName('my app').master('local').getOrCreate()
sc = spark.sparkContext

udataschema = StructType([
   StructField("uid", LongType(), False),
   StructField("mid", LongType(), False),
   StructField("rate",LongType(), False),
   StructField("timestamp", LongType(), False)
 ])

uitemschema = StructType([
  StructField("mid", LongType(), False),
  StructField("title",StringType(), False)
])

data_df = spark.read.format("csv").option("sep", "\t").schema(udataschema).load("./input_week05/u.data")
item_df = spark.read.format("csv").option("sep", "|").schema(uitemschema).load("./input_week05/u.item")

22/10/06 08:07:02 WARN Utils: Your hostname, cherry resolves to a loopback address: 127.0.1.1; using 192.168.0.7 instead (on interface enp6s0)
22/10/06 08:07:02 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/06 08:07:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/10/06 08:07:03 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
item_df.show(10)

+---+--------------------+
|mid|               title|
+---+--------------------+
|  1|    Toy Story (1995)|
|  2|    GoldenEye (1995)|
|  3|   Four Rooms (1995)|
|  4|   Get Shorty (1995)|
|  5|      Copycat (1995)|
|  6|Shanghai Triad (Y...|
|  7|Twelve Monkeys (1...|
|  8|         Babe (1995)|
|  9|Dead Man Walking ...|
| 10|  Richard III (1995)|
+---+--------------------+
only showing top 10 rows



In [5]:
data_df.show(10)

+---+---+----+---------+
|uid|mid|rate|timestamp|
+---+---+----+---------+
|196|242|   3|881250949|
|186|302|   3|891717742|
| 22|377|   1|878887116|
|244| 51|   2|880606923|
|166|346|   1|886397596|
|298|474|   4|884182806|
|115|265|   2|881171488|
|253|465|   5|891628467|
|305|451|   3|886324817|
|  6| 86|   3|883603013|
+---+---+----+---------+
only showing top 10 rows



### 1

In [9]:
# quiz 1
data_filter = data_df.filter(data_df['rate']==5)
joinExpression = data_filter['mid'] == item_df['mid']

data_filter = data_filter.join(item_df, joinExpression, 'inner')
data_filter.filter(substring(data_filter["title"], 0, 1)=='T').select('title').distinct().count()

75

### 2

In [7]:
data_filter = data_df.filter(data_df['rate']==1 | 5)

In [13]:
joinExpression = data_filter['mid'] == item_df['mid']

union_df = data_filter.join(item_df, joinExpression, 'inner').select('title')

In [18]:
union_df.selectExpr('max(length(title))').show()

+------------------+
|max(length(title))|
+------------------+
|                81|
+------------------+

